In [19]:
import sys

sys.path.append('..')

import importcsv

print('Importing csv file...........')
small_dataset = importcsv.opencsv('../processedspeech.csv') 

Importing csv file...........


In [20]:
# remove numbers except 1453 and numbers between 1800-2039 (years)

import re

def remove_numbers(text):
    pattern = re.compile(r'\b(?!(1453 | 18[0-9]{2} | 19[0-9]{2} | 20[0-3][0-9])\b)\d+\b')
    text = re.sub(pattern, '', text)    
    return text

small_dataset['processed_speeches'] = small_dataset['processed_speeches'].apply(remove_numbers)


In [21]:
member_name_list = small_dataset['member_name'].unique()

In [30]:
from word_frequency_calculator import filter_dataframe
from k_decomposition import getkeywords

member_vector = {}
counter = 0
for name in member_name_list:
    counter +=1
    if counter%100 == 0:
        print(counter)
    member_speeches = filter_dataframe(small_dataset, member=name).to_list()
    keywords = getkeywords(member_speeches)
    member_vector[name] = keywords

Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating graph of original data....
Decomposition.....
Creating g

In [31]:
#save the inverted catalog to a json file
import json

# Specify the file path
file_path = "member_ch_vector.json"

# Save the dictionary to a file
with open(file_path, 'w') as file:
    json.dump(member_vector, file)
